[accounts](./accounts.ipynb) | [orders](./orders.ipynb) | [trades](./trades.ipynb) | [positions](./positions.ipynb) | [historical](./historical.ipynb) | [streams](./streams.ipynb) | [errors](./exceptions.ipynb)

# Historical data

OANDA provides access to historical data. The *oandapyV20* has a class to access this data: *oandapyV20.endpoints.instruments.InstrumentsCandles*.

Lets give it a try and download some data for:
  + instrument: EUR_USD
  + granularity: H1
  + from: 2017-01-01T00:00:00

In [6]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from exampleauth import exampleauth

accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)

instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "H1",
    "count": 10,
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))
print(json.dumps(response, indent=2))

Request: v3/instruments/EUR_USD/candles  #candles received: 9
{
  "instrument": "EUR_USD", 
  "candles": [
    {
      "volume": 481, 
      "mid": {
        "h": "1.04712", 
        "c": "1.04662", 
        "l": "1.04572", 
        "o": "1.04577"
      }, 
      "complete": true, 
      "time": "2017-01-03T00:00:00.000000000Z"
    }, 
    {
      "volume": 664, 
      "mid": {
        "h": "1.04808", 
        "c": "1.04758", 
        "l": "1.04646", 
        "o": "1.04665"
      }, 
      "complete": true, 
      "time": "2017-01-03T01:00:00.000000000Z"
    }, 
    {
      "volume": 392, 
      "mid": {
        "h": "1.04780", 
        "c": "1.04721", 
        "l": "1.04709", 
        "o": "1.04761"
      }, 
      "complete": true, 
      "time": "2017-01-03T02:00:00.000000000Z"
    }, 
    {
      "volume": 394, 
      "mid": {
        "h": "1.04848", 
        "c": "1.04848", 
        "l": "1.04715", 
        "o": "1.04718"
      }, 
      "complete": true, 
      "time": "2017-01-0

So, that is 9 records?
... that can be fixed by including the parameter *includeFirst*, see the OANDA documentation for details.

In [8]:
instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "H1",
    "includeFirst": True,
    "count": 10,
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))
print(json.dumps(response, indent=2))

Request: v3/instruments/EUR_USD/candles  #candles received: 10
{
  "instrument": "EUR_USD", 
  "candles": [
    {
      "volume": 974, 
      "mid": {
        "h": "1.04711", 
        "c": "1.04575", 
        "l": "1.04567", 
        "o": "1.04684"
      }, 
      "complete": true, 
      "time": "2017-01-02T23:00:00.000000000Z"
    }, 
    {
      "volume": 481, 
      "mid": {
        "h": "1.04712", 
        "c": "1.04662", 
        "l": "1.04572", 
        "o": "1.04577"
      }, 
      "complete": true, 
      "time": "2017-01-03T00:00:00.000000000Z"
    }, 
    {
      "volume": 664, 
      "mid": {
        "h": "1.04808", 
        "c": "1.04758", 
        "l": "1.04646", 
        "o": "1.04665"
      }, 
      "complete": true, 
      "time": "2017-01-03T01:00:00.000000000Z"
    }, 
    {
      "volume": 392, 
      "mid": {
        "h": "1.04780", 
        "c": "1.04721", 
        "l": "1.04709", 
        "o": "1.04761"
      }, 
      "complete": true, 
      "time": "2017-01-

# Bulk history

## InstrumentsCandles class

It is likely that you want to retrieve more than 10 records. The OANDA docs say that the default number of records
is 500, in case you do not specify. You can specify the number of records to retrieve by using *count*, with a maximum of 5000. The *InstrumentsCandles* class enables you to retrieve the records.

## InstrumentsCandlesFactory

Now if you would like to retrieve a lot of history, you have to make consecutive requests. To make this an easy process the *oandapyV20* library comes with a so called *factory* named *InstrumentsCandlesFactory*.

Using this class you can retrieve all history of an instrument from a certain date. The *InstrumentsCandlesFactory* acts as a generator generating *InstrumentCandles* requests until all data is retrieved. The number of requests can be influenced by specifying *count*. Setting *count* to 5000 would generate a tenth of the requests vs. the default of 500.

Back to our example: lets make sure we request a lot of data, so we set the *granularity* to *M5* and leave the date at 2017-01-01T00:00:00. The will retrieve all records from that date up to today, because we did not specify the *to* parameter. 


In [13]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from exampleauth import exampleauth

accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)

instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "M5",
}

def cnv(r, h):
    # get all candles from the response and write them as a record to the filehandle h
    for candle in r.get('candles'):
        ctime = candle.get('time')[0:19]
        try:
            rec = "{time},{complete},{o},{h},{l},{c},{v}".format(
                time=ctime,
                complete=candle['complete'],
                o=candle['mid']['o'],
                h=candle['mid']['h'],
                l=candle['mid']['l'],
                c=candle['mid']['c'],
                v=candle['volume'],
            )
        except Exception as e:
            print(e, r)
        else:
            h.write(rec+"\n")

datafile = "/tmp/{}.{}.out".format(instrument, params['granularity'])
with open(datafile, "w") as O:
    n = 0
    for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
        rv = client.request(r)
        cnt = len(r.response.get('candles'))
        print("REQUEST: {} {} {}, received: {}".format(r, r.__class__.__name__, r.params, cnt))
        n += cnt
        cnv(r.response, O)
    print("Check the datafile: {} under /tmp!, it contains {} records".format(datafile, n))

REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-02T17:40:00Z', 'from': '2017-01-01T00:00:00Z', 'granularity': 'M5'}, received: 0
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-04T11:25:00Z', 'from': '2017-01-02T17:45:00Z', 'granularity': 'M5'}, received: 436
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-06T05:10:00Z', 'from': '2017-01-04T11:30:00Z', 'granularity': 'M5'}, received: 499
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-07T22:55:00Z', 'from': '2017-01-06T05:15:00Z', 'granularity': 'M5'}, received: 200
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-09T16:40:00Z', 'from': '2017-01-07T23:00:00Z', 'granularity': 'M5'}, received: 222
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCandles {'to': '2017-01-11T10:25:00Z', 'from': '2017-01-09T16:45:00Z', 'granularity': 'M5'}, received: 499
REQUEST: v3/instruments/EUR_USD/candles InstrumentsCan

## ... that was easy ...

All request were made on the default of max. 500 records per request. With a granularity of *M5* this means that we have 288 records per day. The algorithm of the factory does not check on weekends or holidays. Therefore some request only return a part of the 500 requested records because there simply are no more records within the specified timespan.

If you want to decrease the number of requests and increase the number of records returned for a request, just specify *count* as a number higher than 500 and up to max. 5000.

# Create Pandas DataFrame

In [6]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from exampleauth import exampleauth

import pandas as pd 

accountID, access_token = exampleauth.exampleAuth()
client = oandapyV20.API(access_token=access_token)

instrument = "EUR_USD"
params = {
    "from": "2017-01-01T00:00:00Z",
    "granularity": "H1",
    # "count": 10,
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
response = client.request(r)
print("Request: {}  #candles received: {}".format(r, len(r.response.get('candles'))))

candles = {}
for candle in r.response.get('candles'):
    try:
            ohlcv = candle['mid']
            ohlcv['v'] = candle['volume']
            candles[candle['time']] = ohlcv
    except Exception as e:
            print(e)

df = pd.DataFrame.from_dict(candles,orient='index')
df["o"] = pd.to_numeric(df.o, errors='coerce')
df["h"] = pd.to_numeric(df.h, errors='coerce')
df["l"] = pd.to_numeric(df.l, errors='coerce')
df["c"] = pd.to_numeric(df.c, errors='coerce')
df.columns = ['open','high','low','close','volume']
df.index = pd.DatetimeIndex(df.index)
df.index.names = ['Date']
df.info()

df

Request: v3/instruments/EUR_USD/candles  #candles received: 500
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 500 entries, 2017-01-02 23:00:00+00:00 to 2017-01-31 18:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    500 non-null    float64
 1   high    500 non-null    float64
 2   low     500 non-null    float64
 3   close   500 non-null    float64
 4   volume  500 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.4 KB


,open,high,low,close,volume
2017-01-02 23:00:00+00:00,1.04684,1.04711,1.04567,1.04575,974
2017-01-03 00:00:00+00:00,1.04577,1.04712,1.04572,1.04662,481
2017-01-03 01:00:00+00:00,1.04665,1.04808,1.04646,1.04758,664
2017-01-03 02:00:00+00:00,1.04761,1.04780,1.04709,1.04721,392
2017-01-03 03:00:00+00:00,1.04718,1.04848,1.04715,1.04848,394
...,...,...,...,...,...
2017-01-31 14:00:00+00:00,1.07553,1.07950,1.07488,1.07916,4574
2017-01-31 15:00:00+00:00,1.07913,1.08108,1.07796,1.08099,4187
2017-01-31 16:00:00+00:00,1.08096,1.08124,1.07821,1.07933,3253
2017-01-31 17:00:00+00:00,1.07930,1.07969,1.07723,1.07836,1739


# Add cufflink Plot

In [ ]:
%pip install cufflinks

In [4]:
%matplotlib inline 


import warnings
import matplotlib.pyplot as plt 

plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

import cufflinks as cf
import plotly.offline

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [9]:
qf = cf.QuantFig(df, title="EUR_USD", legend='top', name='EUR_USD')
qf.add_sma(periods=20, column='close', color='red')
qf.add_ema(periods=20, color='green')
qf.add_volume()

In [10]:
qf.iplot()